## Segmenting and Clustering Neighborhoods in Toronto

Load required libraries for this task

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
import pandas as pd
import requests

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

In [2]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

Let's define now some helper functions to automatize the extract of html elements 

In [3]:
def find_all(text, sub):
    """ find all occurences of substring in larger string 
    and return their indices relative to the text string
    
    Key arguments
    -------------
    text:      text string
    sub:       substring to be found in text
    
    Return
    -------
    indices:   indices of the found occurences relative to text
    """
    occurencesIndex = []
    index = 0
    while index < len(text):
        index = text.find(sub, index)
        if index == -1:
            return occurencesIndex
        else:
            occurencesIndex.append(index)
            index += 1 
    return occurencesIndex

def getValue(text, b_element, e_element):
    """ get text snippet with given indices for beginning and ending
    Key arguments:
    -------------
    text:       text string
    b_element:  begin html element <element>
    e_element:  end html element </element>
    
    return:
    ------
    text snippet if element found in text, otherwise return original text
    """
    if text.find(b_element) != -1:
        return text[text.find(b_element) + len(b_element):text.find(e_element)]
    else:
        return text
    

The following function extracts the data from the table of the wikipedia homepage 

In [4]:
def getData(text):
    """ get Data from html code
    Key argument:
    ------------
    text:      text from which the data will be extracted (table text)
    
    return:
    -------
    pd.Dataframe 
    """
    rows = zip(find_all(text, "<tr>")[1:], find_all(text, "</tr>")[1:])
    data =  []
    for row_b, row_e in rows:
        row_data = text[row_b:row_e]
        sub_indices = zip(find_all(row_data, "<td>"), find_all(row_data, "</td>"))
        data.append([row_data[sub_b+4:sub_e] for sub_b, sub_e in sub_indices])
    return pd.DataFrame(data, columns = ['PostalCode' , 'Borough', 'Neighborhood'])

table html from the page

In [5]:
tabledata = page[page.find('<table class="wikitable sortable">') + len('<table class="wikitable sortable">'):page.find('<table class="multicol" role="presentation" style="border-collapse: collapse; padding: 0; border: 0; background:transparent; width:100%;">')]

get the data frame of the table html

In [6]:
postcode_can = getData(tabledata)

some decoration since Borough and Neighborhood have data elements of type hyperlink 

In [7]:
postcode_can["Borough"] = [getValue(iString, '">', "</a>") for iString in postcode_can["Borough"]]
postcode_can["Neighborhood"] = [getValue(iString, '">', "</a>").replace("\n", "") for iString in postcode_can["Neighborhood"]]

Only process the cells that have an assigned Borough. Ignore cells with a borough that is Not assigned.

In [8]:
postcode_can = postcode_can[postcode_can["Borough"] != "Not assigned"]

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma.

In [11]:
postcode_can['Neighborhood'] = postcode_can.groupby(['PostalCode'])['Neighborhood'].transform(lambda x: ', '.join(x))
postcode_can = postcode_can[['PostalCode', "Borough", "Neighborhood"]].drop_duplicates()
postcode_can.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront, Regent Park"
6,M6A,North York,"Lawrence Heights, Lawrence Manor"
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,"Rouge, Malvern"
14,M3B,North York,Don Mills North
15,M4B,East York,"Woodbine Gardens, Parkview Hill"
17,M5B,Downtown Toronto,"Ryerson, Garden District"


In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [10]:
postcode_can.shape

(103, 3)